<div style='text-align: right; font-size: 16px;'>
<br/>
<img src="images/HWR_logo.svg" alt="drawing" width="30%"/>
<br/>
Fachbereich Duales Studium Wirtschaft • Technik<br/>
ET1031 Mathematische Grundlagen III (Signale und Systeme)<br/>
Prof. Dr. Luis Fernando Ferreira Furtado, Berlin, 16.09.2025<br/>
</div>

# Kapitel 5. Zeitkontinuierliche LTI-Systeme und die Faltungsintegral

<hr style="border:solid #000000 1px;height:1px;">

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import scipy
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact, FloatSlider, IntSlider
import inspect

# Get the name of the variable used as argument in a function
def varName(var):
    lcls = inspect.stack()[2][0].f_locals
    for name in lcls:
        if id(var) == id(lcls[name]):
            return name
    return None

def show_convolution(t, f, g, t0):

    # Get the name of the functions f and g
    fname = varName(f)
    gname = varName(g)
    
    # Calculate the convolution using the scipy library and the Simpson's rule to compute integral from samples
    convolution = np.zeros(len(t))
    for n, t_ in enumerate(t):
        prod = lambda tau: f(tau) * g(t_-tau)
        convolution[n] = scipy.integrate.simpson(prod(t), t)
    
    # Create the shifted and flipped functions
    f_shift = lambda t: g(t0-t)
    prod = lambda tau: f(tau) * g(t0-tau)
    
    # Prepare the sub-plots
    specs = [[{}, {}], [{"rowspan": 2, "colspan": 2}, None], [None, None], [{"rowspan": 2, "colspan": 2}, None], [None, None]]
    fig = make_subplots(rows=5, cols=2, print_grid=False, specs=specs)

    # Plot the values
    fig.add_trace(go.Scatter(x=t, y=f(t), name=fname+'(t)'), row=1, col=1)
    fig.add_trace(go.Scatter(x=t, y=g(t), name=gname+'(t)'), row=1, col=2)
    fig.add_trace(go.Scatter(x=t, y=f(t), name=fname+'(λ)'), row=2, col=1)
    fig.add_trace(go.Scatter(x=t, y=f_shift(t), name=gname+'(t-λ)'), row=2, col=1)
    fig.add_trace(go.Scatter(x=t, y=prod(t), name=fname+'(λ)'+gname+'(t-λ)', fill='tozeroy'), row=2, col=1)
    fig.add_trace(go.Scatter(x=t, y=convolution, name='y(t)='+fname+'(t)*'+gname+'(t)'), row=4, col=1)
    fig.add_trace(go.Scatter(x=[t0], y=[scipy.integrate.simpson(prod(t), t)], mode='markers', name='y('+str(round(t0,1))+')'), row=4, col=1)

    # Update the layout
    fig.update_layout(height=600, title_text='Faltungsintegral y(t) = '+fname+'(t)*'+gname+'(t)', template='plotly_white', xaxis=dict(title_text='t'))
    tickvals = np.arange(int(round(min(t))), int(round(max(t)))+1)
    ticktext = [str(tick) for tick in tickvals]
    ticktext[-2] = 't'
    fig.update_xaxes(tickmode='array', tickvals=tickvals, ticktext=ticktext)
    ticktext[-2] = 'λ'
    fig['layout']['xaxis3']['ticktext'] = ticktext

    # Show the figure
    fig.show()

def step(t):
    """
    ------------------------------------------------------------------------------------------------------------
    [Continuous Time]
    This function calculates an approximation of the step function x(t) = u(t)

    Parameter:
        t: a np.ndarray that represents the continuous time axis

    Return:
        x: a np.ndarray with the step signal x(t) = u(t)
    ------------------------------------------------------------------------------------------------------------
    """  

    x = np.zeros(len(t))
    for k, tt in enumerate(t):
        if tt >= 0:
            x[k] = 1.0

    return x    


def rect(t, tau):
    """
    ------------------------------------------------------------------------------------------------------------
    [Continuous Time]
    This function calculates an approximation of the rectangular function x(t) = ∏(t/tau)

    Parameters:
        t: a np.ndarray that represents the continuous time axis
        tau: a float with the width of the rectangular pulse

    Return:
        x: a np.ndarray with the rectangular signal x(t) = ∏(t/tau)
    ------------------------------------------------------------------------------------------------------------
    """          

    x = np.zeros(len(t))
    for k, tk in enumerate(t):
        if np.abs(tk) > tau/2.:
            x[k] = 0
        else:
            x[k] = 1

    return x

def tri(t, tau):
    """
    ------------------------------------------------------------------------------------------------------------
    [Continuous Time]
    This function calculates an approximation of the triangular function x(t) = ∧(t/tau)

    Parameters:
        t: a np.ndarray that represents the continuous time axis
        tau: a float with the width of the half of the triangular pulse

    Return:
        x: a np.ndarray with the triangular signal x(t) = ∧(t/tau)
    ------------------------------------------------------------------------------------------------------------   
    """          

    x = np.zeros(len(t))
    for k, tk in enumerate(t):
        if np.abs(tk) > tau/1.:
            x[k] = 0
        else:
            x[k] = 1 - np.abs(tk)/tau

    return x


<hr style="border:solid #000000 1px;height:1px;">

#### Faltungsintegral: 
__Beispiel__
<br/>
$x(t) = \bigwedge(t)$<br/>
$h(t) = u(t)e^{-2t}$<br/>
$$y(t)=x(t)*h(t)=\int x(λ)h(t-λ)dλ$$

In [ ]:
def update(t0):

    # Set the axis t and the two functions to be convoluted
    t = np.arange(-5, 5 + 0.01, 0.01)
    x = lambda t: np.maximum(0, 1-abs(t))
    h = lambda t: (t>0) * np.exp(-2*t)
    
    show_convolution(t, x, h, t0)

t0 = FloatSlider(min=-4.0, max=5.0, value=-2.0, step=0.1, description='t')
interact(update, t0=t0);

<hr style="border:solid #000000 1px;height:1px;">

#### Faltungsintegral: 
__Beispiel 5.1__
<br/>Betrachten Sie das Eingangssignal $x(t)$ und die Impulsantwort $h(t)$ und berechnen Sie das Faltungsintegral $y(t)=x(t)*h(t)$ : <br/><br/>
$x(t) = 2\prod(t-3,5)$ <br/><br/>
$h(t) = \prod\left(\frac{t-1}{2}\right)$

In [ ]:
def update(t0):

    # Set the axis t and the two functions to be convoluted
    t = np.arange(-1, 8 + 0.01, 0.01)
    x = lambda t: (t>3)*(t<4)*2
    h = lambda t: (t>0)*(t<2)*1
    #x = lambda t: 2 * rect(t - 3.5, 1)
    #h = lambda t: rect(t - 1, 2)    
    
    show_convolution(t, x, h, t0)

t0 = FloatSlider(min=2.0, max=7.0, value=2.0, step=0.1, description='t')
interact(update, t0=t0);

<hr style="border:solid #000000 1px;height:1px;">

#### Faltungsintegral: 
__Beispiel 5.2__
<br/>
Betrachten Sie das Eingangssignal $x(t)$ und die Impulsantwort $h(t)$ und berechnen Sie das Faltungsintegral $y(t)=x(t)*h(t)$ : <br/><br/>
$x(t) = 2(t-3)\prod(t-3,5)$<br/><br/>
$h(t) = \prod\left(\frac{t-1}{2}\right)$

In [ ]:
def update(t0):

    # Set the axis t and the two functions to be convoluted
    t = np.arange(-1, 8 + 0.01, 0.01)
    x = lambda t: (t>3)*(t<4)*2*(t-3)
    h = lambda t: (t>0)*(t<2)*1
    #x = lambda t: 2 * (t -3) * rect(t - 3.5, 1)
    #h = lambda t: rect(t - 1, 2)    
    
    show_convolution(t, x, h, t0)

t0 = FloatSlider(min=2.0, max=7.0, value=2.0, step=0.1, description='t')
interact(update, t0=t0);

<hr style="border:solid #000000 1px;height:1px;">

#### Faltungsintegral: 
__Beispiel 5.3__
<br/>
Betrachten Sie das Eingangssignal $x(t)$ und die Impulsantwort $h(t)$ und berechnen Sie das Faltungsintegral $y(t)=x(t)*h(t)$ : <br/><br/>
$x(t) = sin( \pi t) u(t)$<br/><br/>
$h(t) = \frac{1}{τ}\prod\left(\frac{t-τ/2}{τ}\right)$

In [ ]:
def update(t0, tau):
    
    # Set the axis t and the two functions to be convoluted
    t = np.arange(-2, 8 + 0.01, 0.01)
    h = lambda t: (t>0) * (t<tau) * 1/tau
    x = lambda t: (t>0) * np.sin(np.pi*t)
    #x = lambda t: rect(t - tau/2, tau) * 1/tau
    #h = lambda t: step(t) * np.sin(np.pi*t)

    show_convolution(t, x, h, t0)

t0 = FloatSlider(min=-1.0, max=7.0, value=-1.0, step=0.1, description='t')
tau = FloatSlider(min=2.0, max=6.0, value=4.0, step=0.1, description='τ')
amplitude = FloatSlider(min=0.5, max=2.0, value=0.5, step=0.1, description='A')

interact(update, t0=t0, tau=tau);

<hr style="border:solid #000000 1px;height:1px;">

#### Faltungsintegral: 
__Hausaufgabe 4, Aufgabe 2__
<br/>
Betrachten Sie das Eingangssignal $x(t)$ und die Impulsantwort $h(t)$ und berechnen Sie das Faltungsintegral $y(t)=x(t)*h(t)$ : <br/><br/>
$x(t) = 3\prod\left(\frac{t-2}{4}\right)$<br/><br/>
$h(t) = 2\bigwedge\left(\frac{t}{2}\right)u(t)=-(t-2)\prod\left(\frac{t-1}{2}\right)$

In [ ]:
def update(t0):

    # Set the axis t and the two functions to be convoluted
    t = np.arange(-1, 8 + 0.01, 0.01)
    x = lambda t: (t>0)*(t<4)*3
    h = lambda t: (t>0)*(t<2)*(-t+2)
    #x = lambda t: rect(t-2,4)
    #h = lambda t: 2 * tri(t, 2) * rect(t)
    
    show_convolution(t, x, h, t0)

t0 = FloatSlider(min=2.0, max=7.0, value=2.0, step=0.1, description='t')
interact(update, t0=t0);

<hr style="border:solid #000000 1px;height:1px;">

#### Faltungsintegral: 
__Hausaufgabe 4, Aufgabe 3__
<br/>
Betrachten Sie das Eingangssignal $x(t)$ und die Impulsantwort $h(t)$ und berechnen Sie das Faltungsintegral $y(t)=x(t)*h(t)$ : <br/><br/>
$x(t) = \prod\left(\frac{t-τ/2}{τ}\right)$<br/><br/>
$h(t) = e^{-∝t}u(t)$

In [ ]:
def update(t0, tau, alpha):
    
    # Set the axis t and the two functions to be convoluted
    t = np.arange(-4, 10 + 0.01, 0.01)
    x = lambda t: (t>0)*(t<tau)*1
    h = lambda t: (t>0)*np.exp(-alpha*t)
    #x = lambda t: rect(t - tau/2, tau)  
    #h = lambda t: step(t) * np.exp(-alpha * t)
    
    show_convolution(t, x, h, t0)

t0 = FloatSlider(min=-1.0, max=7.0, value=-1.0, step=0.1, description='t')
tau = FloatSlider(min=2.0, max=6.0, value=4.0, step=0.1, description='τ')
alpha = FloatSlider(min=0.0, max=2.0, value=1.0, step=0.1, description='∝')

interact(update, t0=t0, tau=tau, alpha=alpha);